In [1]:
import numpy as np
print(np.__version__)
print(hasattr(np, "ctypeslib"))
print(hasattr(np, "broadcast_to"))

1.26.4
True
True


In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

a = np.array([[1, 2]])
b = np.array([[2, 3]])
print(cosine_similarity(a, b))  # → [[0.9926...]]


[[0.99227788]]


In [5]:
import os
import json
import numpy as np
from langchain.embeddings import AzureOpenAIEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))
from config import settings



In [7]:
# 1. 특성 불러오기
load_path = os.path.abspath(os.path.join(os.getcwd(), "../../test/autotextrecognation/resources/llm_features.json"))
with open(load_path, "r", encoding="utf-8") as f:
    features = json.load(f)

feature_everyday = features["everyday"]
feature_work = features["work"]


In [8]:
# 2. Azure OpenAI 임베딩 모델 초기화
embedding_model = AzureOpenAIEmbeddings(
    azure_deployment=settings.EMBEDDING_DEPLOYMENT,
    openai_api_key=settings.EMBEDDING_API_KEY,
    openai_api_version=settings.EMBEDDING_API_VERSION,
    azure_endpoint=settings.EMBEDDING_ENDPOINT,
    chunk_size = settings.EMBEDDING_CHUNK_SIZE
)



/var/folders/wy/_jj74ny11pd8y1qwn7xj5vc80000gn/T/ipykernel_23831/3472893788.py:2: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embedding_model = AzureOpenAIEmbeddings(


In [9]:
# 3. 특성 벡터 임베딩
vector_everyday = embedding_model.embed_query(feature_everyday)
vector_work = embedding_model.embed_query(feature_work)



In [10]:
# 4. 분류 함수 정의
def classify_by_similarity(text: str) -> str:
    text_vector = embedding_model.embed_query(text)

    # Cosine 유사도 계산
    sim_everyday = cosine_similarity([text_vector], [vector_everyday])[0][0]
    sim_work = cosine_similarity([text_vector], [vector_work])[0][0]

    classification = "일상" if sim_everyday > sim_work else "업무"

    print(f"문장: {text}")
    print(f"🟦 일상 유사도: {sim_everyday:.4f}")
    print(f"🟥 업무 유사도: {sim_work:.4f}")
    print(f"✅ 분류 결과: {classification}\n")

    return classification

# 5. 테스트 문장
test_sentences = [
    "오늘 점심에 뭐 먹지?",
    "포트폴리오에 기술 스택을 잘 정리해 주세요.",
    "날씨가 좋아서 산책 다녀왔어.",
    "이번 프로젝트 마감은 언제인가요?",
]

for sentence in test_sentences:
    classify_by_similarity(sentence)


문장: 오늘 점심에 뭐 먹지?
🟦 일상 유사도: 0.7899
🟥 업무 유사도: 0.7232
✅ 분류 결과: 일상

문장: 포트폴리오에 기술 스택을 잘 정리해 주세요.
🟦 일상 유사도: 0.7722
🟥 업무 유사도: 0.8085
✅ 분류 결과: 업무

문장: 날씨가 좋아서 산책 다녀왔어.
🟦 일상 유사도: 0.8019
🟥 업무 유사도: 0.7276
✅ 분류 결과: 일상

문장: 이번 프로젝트 마감은 언제인가요?
🟦 일상 유사도: 0.7721
🟥 업무 유사도: 0.7611
✅ 분류 결과: 일상

